# AWS - RDS MySQL
### Paquetes requeridos
Librería de Python necesaria para establecer conexión a una base de datos MySQL

In [ ]:
# !pip install "pymysql==1.0.2"

In [ ]:
# !pip install sqlalchemy

In [ ]:
# !pip install mysql-connector-python

### Credenciales
Para acceder a la BD necesitaremos tres datos:
1. Endpoint/host: la dirección donde se encuentra el servidor
2. Usuario
3. Contraseña

In [1]:
host = "database-1.c9mwoowk2nth.eu-north-1.rds.amazonaws.com"
username = "admin"
password = "12345678"
port = 3306

### Conexion BD

In [2]:
import pymysql

In [3]:
'''
pymysql.cursors.Dictcursor para que los resultados que devuelva sean diccionarios,
por defecto devuelve tuplas. Asi podemos acceder por clave a las columnas
'''

db = pymysql.connect(host = host,
                     user = username,
                     password = password,
                     cursorclass = pymysql.cursors.DictCursor
                     )
# El objeto cursor es el que ejecutará las queries y devolverá los resultados

cursor = db.cursor()

### Version DB

In [ ]:
'''
Este es el engine version de la BD de AWS
fechtone trae la primera linea de la consulta-
El execute() devuelve el numero de filas a las que ha afectado la query,
en este caso, devuelve una unica fila.

Execute se guarda en la conexion pero hasta que no hacemos commit
no se ejecutan las queries que modifiquen la base de datos de alguna manera:
de insert de datos, creación de tablas, etc 
'''

cursor.execute("SELECT VERSION()")
version = cursor.fetchone()
print (f"MysQL version: {version}")

### Creación de DB

In [ ]:
# Creamos una BD. Tenemos una instancia de MySQL, pero no una BD
# Los comandos de SQL se suelen poner en mayuscula
create_db = '''CREATE DATABASE country_database'''
cursor.execute(create_db)

# Podemos eliminar la BD
# drop_db = '''DROP DATABASE country_database'''
# cursor. execute(drop_db)
# La volvemos a crear
# cursor.execute(create_db)
# el output es el numero de filas afectadas

In [ ]:
cursor.execute("SHOW DATABASES")
cursor.fetchall()

### Creación de tablas

In [ ]:
'''
Creamos una tabla 
Esto da error porque no le hemos dicho al servidor que BD
queremos usar
'''

# Crear una tabla que se llame country
create_table = '''
    CREATE TABLE country (
        id INT NOT NULL auto_increment,
        name TEXT,
        continent TEXT,
        population INT,
        gdp DOUBLE,
        primary key (id)
    )
'''
cursor.execute(create_table)

### Seleccionar la BD

In [ ]:
# Para usar la BD recien creada
cursor.connection.commit()
use_db = '''USE country_database'''
cursor.execute(use_db)

In [ ]:
# Chequear todas las tablas que tiene mi db
cursor.execute('SHOW TABLES')
cursor.fetchall()

In [ ]:
# Si queremos borrar alguna tabla 
# drop_table = '''DROP TABLE country'''
# cursor.execute(drop_table)

### Insertar datos

In [ ]:
insert_data = '''
INSERT INTO country(name, continent, population, gdp)
VALUES ('Spain','Europe', 48, 1.62)
'''
cursor.execute(insert_data)

In [20]:
name = "Italy"
continent = "Europe"
population = 59
gdp = 2.3

In [21]:
insert_data = f'''
INSERT INTO country(name,continent,population,gdp) VALUES ("{name}","{continent}",{population},{gdp})
'''

In [ ]:
cursor.execute(insert_data)

### Leer datos

In [ ]:
query = '''
SELECT * FROM country WHERE name LIKE "%pa%"
'''
cursor.execute(query)

In [ ]:
print(cursor.fetchall())

### Guardar los cambios
Hay que ejecutar el commit antes de cerrar la sesión de la BD para que se guarden todos los cambios

In [25]:
db.commit()

### Excepciones
Se recomienda rodear este tipo de sentencias con un try/except, ya que suele ser software productivo y tiene que saber manejar fallos

In [48]:
try:
    sql = '''SELECT * FROM country'''
    cursor.execute(sql)
    cursor.fetchall()
except Exception as e:
    print(e)

### Insertar datos de un CSV

In [10]:
# Creamos el CSV
import pandas as pd

df= pd.read_csv('country_data.csv', sep=';')

In [ ]:
df

In [11]:
for datos in zip(*[valor.values() for clave,valor in df.to_dict().items()]):
    insert = "INSERT INTO country(name,continent,population,gdp) VALUES ('%s','%s',%d,%.1f)" \
    %(datos[0],datos[1],datos[2],datos[3])
    cursor.execute(insert)

In [ ]:
query = "SELECT * FROM country"
cursor.execute(query)
cursor.fetchall()

In [15]:
# Utilizando sqlalchemy
from sqlalchemy import create_engine
# import pymysql

engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(user = username, pw = password, host = host, db = "country_database"))

In [ ]:
df.to_sql(name = "country", con= engine, if_exists= "append", index = False)

In [18]:
db.commit()

### Cerrar la conexión cuando acabemos

In [20]:
db.close()

In [21]:
conn = engine.connect()
conn.close()